In [ ]:
'''
Steps to be completed as a prerequisite -> 
(1) Download the 3 data files and place them in proper location (Maybe create a data engineering folder)
(2) Open MySQL Workbemch and login and your server status should be running
(3) Open up a new Jupyter Notebook and type the below code
'''

In [ ]:
# Importing important libraries
import pandas as pd
import pymysql

: 

In [2]:
# MySQL Connection Setup 
connection = pymysql.connect(
    host = 'localhost', 
    user = 'root',
    password = 'demo@123', # Use your own password here
    autocommit = True
)
cursor = connection.cursor()

In [3]:
# Getting the cursor details
print(cursor)

In [4]:
# Creating the "BigMart" database 
cursor.execute("CREATE DATABASE IF NOT EXISTS BigMart;")
cursor.execute("USE BigMart;")

0

In [ ]:
'''
Please refresh you MySQL Schemas and verify if you are able to get "Bigmart" database.
'''

In [7]:
# Loading the 3 xml files
df_item = pd.read_xml("C:/Users/jaipu/Documents/Black_Elephant/Weekend_January/Data_Engineering_Project/df_item.xml")
df_outlet = pd.read_xml("C:/Users/jaipu/Documents/Black_Elephant/Weekend_January/Data_Engineering_Project/df_outlet.xml")
df_sales = pd.read_xml("C:/Users/jaipu/Documents/Black_Elephant/Weekend_January/Data_Engineering_Project/df_sales.xml")

In [8]:
# Creating function for creating tables 
def create_table_from_df(df, table_name):
    cols = []
    for col in df.columns:
        dtype = df[col].dtype
        if 'int' in str(dtype):
            sql_type = 'INT'
        elif 'float' in str(dtype):
            sql_type = 'FLOAT'
        else:
            sql_type = 'VARCHAR(255)'
        cols.append(f"{col} {sql_type}")

    columns_sql = ", ".join(cols)
    create_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql})"
    cursor.execute(create_sql)

In [9]:
# Creating table in database
create_table_from_df(df_item, "item_info")
create_table_from_df(df_outlet, "outlet_info")
create_table_from_df(df_sales, "sales_info")

In [ ]:
# Please confirm if the tables got created in MySQL Database "Bigmart"

In [15]:
# Creating a function to insert the data into tables 
def insert_data(df, table_name):
    df = df.where(pd.notnull(df), None) # Replace NaN entry with NULL 
    cols = ",".join([f"`{col}`" for col in df.columns])
    placeholders = ",".join(["%s"] * len(df.columns))
    insert_sql = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"
    data = [tuple(row) for row in df.itertuples(index = False, name = None)]
    cursor.executemany(insert_sql, data)

In [16]:
# Inserting the data into MySQL Tables
insert_data(df_item, 'item_info')
insert_data(df_outlet, 'outlet_info')
insert_data(df_sales, 'sales_info')

In [ ]:
# Please check in MySQL whether we have correctly updated the data or not.

In [17]:
# Closing the connection 
cursor.close()
connection.close()